In [22]:
import pandas as pd
import numpy as np

In [29]:
asg = pd.read_csv("data/ASG_Dates.csv")

In [30]:
att = pd.read_csv("data/Attendance_and_Capacity_Data.csv")

In [32]:
dma = pd.read_csv("data/DMA_Households.csv")

In [34]:
end = pd.read_csv("data/End_of_Season_WPct.csv")

In [6]:
gam = pd.read_csv("data/Game_Data.csv")

In [7]:
jer = pd.read_csv("data/Jersey_Sales_Rankings_Data.csv")

In [8]:
nat = pd.read_csv("data/National_Ratings.csv")
# lea = pd.read_csv("data/League_EV_Data.xlsx")

In [9]:
pct = pd.read_csv("data/Pct_Indy_Tix_Sold.csv")

//anaconda/envs/python2env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
pla = pd.read_csv("data/Player_Data.csv")

In [11]:
rsn = pd.read_csv("data/RSN_Ratings.csv")

In [12]:
tea = pd.read_csv("data/Team_Abbreviations.csv")

In [13]:
web = pd.read_csv("data/Web_Metrics.csv")

In [14]:
wpc = pd.read_csv("data/WPct_by_Day.csv")

In [15]:
pct.head()

,Home,Away,Game_Date,Current_Date,Pct_Indy_Tix_Sold
0,DAL,WAS,2010-10-05,2010-09-07,0
1,DAL,WAS,2010-10-05,2010-09-08,0.125412541
2,DAL,WAS,2010-10-05,2010-09-10,0.132013201
3,DAL,WAS,2010-10-05,2010-09-11,0.184818482
4,DAL,WAS,2010-10-05,2010-09-12,0.198019802


In [19]:
pct.types()

AttributeError: 'DataFrame' object has no attribute 'types'

In [28]:
dma.isnull()

,Season,ATL,BOS,BRK,CHA,CHI,CLE,DAL,DEN,DET,...,OKC,ORL,PHI,PHX,POR,SAC,SAS,TOR,UTA,WAS
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [26]:
pct['Current_Date'].isnull()

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
454009    False
454010    False
454011    False
454012    False
454013    False
454014    False
454015    False
454016    False
454017    False
454018    False
454019    False
454020    False
454021    False
454022    False
454023    False
454024    False
454025    False
454026    False
454027    False
454028    False
454029    False
454030    False
454031    False
454032    False
454033    False
454034    False
454035    False
454036    False
454037    False
454038    False
Name: Current_Date, Leng